# Assignment: Build a Semantic Kernel Bot with Weather and News Skills


---

### Objective:
This assignment will guide you through building a **Semantic Kernel (SK) bot** that can answer user queries by executing specialized "skills" (also known as plugins or native functions). You will create two such skills: one for fetching **weather information** and another for retrieving **news articles**. This demonstrates how SK enables LLMs to leverage external tools and data sources to perform complex, real-world tasks.

---

### Instructions:
1.  **LLM Access**: You will need access to an LLM API. **OpenAI's models (e.g., GPT-4o, GPT-4, GPT-3.5-turbo)** are recommended for their robust function calling capabilities, which Semantic Kernel relies on. You can also use Azure OpenAI or Google Gemini.
2.  **External API Keys**: To build functional skills, you'll need API keys for external services:
    * **Weather**: Obtain a free API key from [OpenWeatherMap](https://openweathermap.org/api) (Current Weather Data API is sufficient).
    * **News**: Obtain a free API key from [NewsAPI](https://newsapi.org/) (Developer API is sufficient for non-commercial use).
3.  **Environment Setup**: Install the necessary Python library: `pip install semantic-kernel python-dotenv requests`.
4.  **API Key Management**: Securely handle all your API keys. It's **highly recommended** to use environment variables and `python-dotenv` for this. Create a `.env` file in your project root.
5.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
6.  **Error Handling**: Implement basic `try-except` blocks within your skill functions to handle potential API call failures gracefully.
7.  **Analysis**: Critically evaluate your bot's performance, skill orchestration, and limitations.

---

## Part 1: Setup and LLM/API Configuration
Begin by installing necessary libraries and configuring your LLM and external API keys.

### Task 1.1: Install Libraries and Load Environment Variables
Install `semantic-kernel` and `python-dotenv`. Load your API keys from a `.env` file. You'll also need `requests` for API calls.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install semantic-kernel python-dotenv requests --quiet

import os
import semantic_kernel as sk
from semantic_kernel.functions import kernel_function # Changed from sk_function for newer SK versions
from semantic_kernel.connectors.ai.open_ai import OpenAIClient, OpenAIChatCompletion
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# --- IMPORTANT: Create a .env file in the same directory as this notebook with the following lines: ---
# OPENAI_API_KEY="YOUR_OPENAI_API_KEY_HERE"
# OPENWEATHERMAP_API_KEY="YOUR_OPENWEATHERMAP_API_KEY_HERE"
# NEWSAPI_API_KEY="YOUR_NEWSAPI_API_KEY_HERE"

# Initialize the kernel
kernel = sk.Kernel()

# Configure LLM (OpenAI example)
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    # Use OpenAIChatCompletion for chat-based models
    kernel.add_service(
        OpenAIChatCompletion(
            service_id="chat-gpt",
            ai_model_id="gpt-4o", # Or "gpt-4", "gpt-3.5-turbo"
            api_key=api_key
        ),
    )
    print("Semantic Kernel initialized with OpenAI LLM!")
else:
    print("WARNING: OPENAI_API_KEY not found in environment variables. Please set it in .env file.")

print("Libraries loaded and environment setup complete!")

---

## Part 2: Define Native Skills (Plugins)
Create Python classes that encapsulate your weather and news functionalities. Each method that should be exposed to the LLM will be decorated with `@kernel_function` (or `@sk_function` depending on your SK version).

### Task 2.1: `WeatherPlugin`
Create a class `WeatherPlugin` with a method `get_current_weather`. This method should make an API call to OpenWeatherMap to fetch current weather data.

* **Method Signature**: `@kernel_function(description="Gets the current weather for a specified location.")`
    * `location`: `str` (required), description="The city and country (e.g., 'London, UK') to get weather for."
    * `unit`: `str` (optional, default='celsius'), description="Temperature unit: 'celsius' or 'fahrenheit'."
* **API Call**: Use `requests` to call OpenWeatherMap's Current Weather Data API.
    * Base URL: `https://api.openweathermap.org/data/2.5/weather`
    * Parameters: `q={location}`, `appid={API_KEY}`, `units={metric/imperial}`.
* **Return Value**: A string summarizing the weather (e.g., "Current weather in London: 15°C, cloudy.") or an error message if the API call fails.
* **Error Handling**: Implement `try-except` for network errors or API response issues.

In [ ]:
class WeatherPlugin:
    def __init__(self):
        self.api_key = os.getenv("OPENWEATHERMAP_API_KEY")
        self.base_url = "https://api.openweathermap.org/data/2.5/weather"

    @kernel_function(
        description="Gets the current weather for a specified location.",
        name="get_current_weather"
    )
    @sk.kernel_function.input("location", description="The city and country (e.g., 'London, UK') to get weather for.")
    @sk.kernel_function.input("unit", description="Temperature unit: 'celsius' or 'fahrenheit'. Defaults to celsius.", default_value="celsius")
    async def get_current_weather(self, location: str, unit: str = "celsius") -> str:
        if not self.api_key:
            return "Weather API key not configured."

        units_param = "metric" if unit.lower() == "celsius" else "imperial"

        params = {
            "q": location,
            "appid": self.api_key,
            "units": units_param
        }

        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
            data = response.json()

            if data.get("cod") == "404":
                return f"Location '{location}' not found."

            main = data.get("main", {})
            weather_desc = data.get("weather", [{}])[0].get("description", "")
            temp = main.get("temp")
            humidity = main.get("humidity")

            if temp is not None:
                return f"Current weather in {location}: {temp}°{unit.upper()[0]}, {weather_desc}, humidity: {humidity}%."
            else:
                return f"Could not retrieve temperature for {location}."
        except requests.exceptions.RequestException as e:
            return f"Error fetching weather for {location}: {e}"
        except Exception as e:
            return f"An unexpected error occurred while processing weather data: {e}"

print("WeatherPlugin created!")

### Task 2.2: `NewsPlugin`
Create a class `NewsPlugin` with a method `get_latest_news`. This method should make an API call to NewsAPI to fetch recent news articles on a given topic.

* **Method Signature**: `@kernel_function(description="Gets the latest news articles for a specified topic.")`
    * `topic`: `str` (required), description="The news topic or keyword to search for."
    * `count`: `int` (optional, default=3), description="The number of news articles to retrieve."
* **API Call**: Use `requests` to call NewsAPI's `everything` endpoint.
    * Base URL: `https://newsapi.org/v2/everything`
    * Parameters: `q={topic}`, `apiKey={API_KEY}`, `pageSize={count}`.
* **Return Value**: A formatted string listing the news articles (title, source, URL) or an error message.
* **Error Handling**: Implement `try-except` for network errors or API response issues.

In [ ]:
class NewsPlugin:
    def __init__(self):
        self.api_key = os.getenv("NEWSAPI_API_KEY")
        self.base_url = "https://newsapi.org/v2/everything"

    @kernel_function(
        description="Gets the latest news articles for a specified topic.",
        name="get_latest_news"
    )
    @sk.kernel_function.input("topic", description="The news topic or keyword to search for.")
    @sk.kernel_function.input("count", description="The number of news articles to retrieve. Defaults to 3.", default_value=3, type=int)
    async def get_latest_news(self, topic: str, count: int = 3) -> str:
        if not self.api_key:
            return "News API key not configured."

        if count > 5: # Limit to avoid excessive API calls on free tier
            count = 5

        params = {
            "q": topic,
            "apiKey": self.api_key,
            "pageSize": count,
            "language": "en" # Limit to English news
        }

        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            data = response.json()

            articles = data.get("articles", [])
            if not articles:
                return f"No news found for topic '{topic}'."

            news_summary = []
            for i, article in enumerate(articles):
                title = article.get("title", "No Title")
                source = article.get("source", {}).get("name", "Unknown Source")
                url = article.get("url", "")
                news_summary.append(f"{i+1}. {title} (Source: {source}) - {url}")

            return f"Latest news on '{topic}':\n" + "\n".join(news_summary)
        except requests.exceptions.RequestException as e:
            return f"Error fetching news for '{topic}': {e}"
        except Exception as e:
            return f"An unexpected error occurred while processing news data: {e}"

print("NewsPlugin created!")

---

## Part 3: Integrate Skills and Build the Bot
Add your newly created `WeatherPlugin` and `NewsPlugin` instances to the Semantic Kernel and set up an interaction loop.

### Task 3.1: Import Skills into the Kernel
Instantiate your plugin classes and import them into the `kernel`.

In [ ]:
weather_skill = kernel.import_plugin_from_object(WeatherPlugin(), plugin_name="WeatherPlugin")
news_skill = kernel.import_plugin_from_object(NewsPlugin(), plugin_name="NewsPlugin")

print("WeatherPlugin and NewsPlugin imported into the kernel!")

### Task 3.2: Create the Bot Interaction Loop
Define an asynchronous function `chat_with_bot` that takes user input, uses `kernel.invoke()`, and prints the bot's response. This will be your main interaction point.

In [ ]:
async def chat_with_bot(query: str) -> str:
    print(f"\nUser: {query}")
    try:
        # The kernel will automatically decide which skill/function to use based on the query
        result = await kernel.invoke(query)
        # For simpler queries, the result will directly be the LLM's response
        # For tool calls, the LLM will orchestrate and use the tool, then respond

        # Newer SK versions return a FunctionResult or ChatMessageContent
        if hasattr(result, 'value'): # Check if it's an old style result object
            response_content = str(result.value)
        elif hasattr(result, 'text'): # Check if it's a ChatMessageContent
            response_content = result.text
        else:
            response_content = str(result)

        print(f"Bot: {response_content}")
        return response_content
    except Exception as e:
        print(f"Bot (Error): An error occurred during processing: {e}")
        return f"Sorry, I encountered an error: {e}"

print("chat_with_bot function defined!")

---

## Part 4: Test and Evaluate the Bot
Run various queries to test your bot's ability to use the weather and news skills. Observe the console output carefully to see how the LLM orchestrates the skill calls.

### Task 4.1: Test with Weather Queries
Run the `chat_with_bot` function with queries that require the `WeatherPlugin`.

In [ ]:
import asyncio

# Test Case 1: Simple weather query
await chat_with_bot("What's the weather like in London, UK?")

# Test Case 2: Weather with unit specification
await chat_with_bot("Tell me the temperature in Paris, France in Fahrenheit.")

# Test Case 3: Unknown location
await chat_with_bot("How's the weather in Narnia?")

### Task 4.2: Test with News Queries
Run the `chat_with_bot` function with queries that require the `NewsPlugin`.

In [ ]:
# Test Case 4: Simple news query
await chat_with_bot("Give me the latest news on AI.")

# Test Case 5: News with specific count
await chat_with_bot("Find 2 recent news articles about electric vehicles.")

# Test Case 6: No news found
await chat_with_bot("What's the news about sentient potatoes?")

### Task 4.3: Test with Mixed/Complex Queries
Challenge your bot with queries that might require a combination of general knowledge and skill execution, or even multiple skill calls (though SK's default chaining might require explicit prompting for multi-step tasks).

In [ ]:
# Test Case 7: Mixed query (might only use one skill or general knowledge)
await chat_with_bot("What's the current weather in Tokyo and any recent news about technology companies?")

# Test Case 8: Query that requires general LLM response
await chat_with_bot("What is the capital of France?")

---

## Part 5: Analysis and Reflection
Based on your testing, answer the following questions.

### Task 5.1: Skill Orchestration and Intent Recognition
* **Effectiveness**: How effectively did Semantic Kernel (and the underlying LLM) determine which skill to call based on your queries? Provide examples where it succeeded and where it might have struggled.
* **Parameter Extraction**: Did the LLM correctly extract parameters (like `location`, `topic`, `unit`, `count`) from your natural language queries and pass them to the skills?
* **Fallback Behavior**: How did the bot respond to queries that did not align with any of its defined skills? Did it provide a reasonable general LLM response?

### Task 5.2: Tool Integration and Data Processing
* **API Integration**: Describe the process of integrating external APIs (OpenWeatherMap, NewsAPI) as native functions within Semantic Kernel. What are the advantages of this approach?
* **Data Presentation**: How well did your skill functions process the raw API responses and present them in a user-friendly format in the bot's output?
* **Error Handling**: Did your `try-except` blocks effectively catch and report errors from the external APIs? Provide an example of an error message you observed.

### Task 5.3: Limitations and Future Improvements
* **Current Limitations**: What are the current limitations of your bot? (e.g., restricted to specific locations/topics, limited number of news articles, no support for complex chained requests).
* **Future Enhancements**: If you were to extend this bot, what additional skills or functionalities would you add? How would you improve its robustness or user experience?
* **Prompt Engineering vs. Skills**: Discuss how skills (native functions) enhance the LLM's capabilities beyond pure prompt engineering for tasks requiring external, real-time data or specific actions.

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Make sure your `.env` file (or equivalent API key setup) is mentioned but **NOT** included in the submitted notebook for security reasons.
* Save your Jupyter Notebook as `[YourName]_SemanticKernel_Skills_Assignment.ipynb`.